## Transform Refunds Data
1. Extract specific portion of the string from refund_reason using split function
1. Extract specific portion of the string from refund_reason using regexp_extract function
1. Extract date and time from the refund_timestamp
1. Write transformed data to the Silver schema in hive metastore [Default Location: /user/hive/warehouse]

In [0]:
SELECT refund_id,
       payment_id,
       refund_timestamp,
       refund_amount,
       refund_reason
from hive_metastore.bronze.refunds;

### 1. Extract specific portion of the string from refund_reason using split function
> [Documentation for split Function](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/split)

In [0]:
SELECT refund_id,
       payment_id,
       refund_timestamp,
       refund_amount,
       SPLIT(refund_reason, ":")[0] AS refund_reason,
       SPLIT(refund_reason, ":")[1] AS refund_source
from hive_metastore.bronze.refunds;

### 2. Extract specific portion of the string from refund_reason using regexp_extract function
> [Documentation for regexp_extract Function](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/regexp_extract)  
> [Regex Pattern](https://docs.oracle.com/javase/7/docs/api/java/util/regex/Pattern.html)

In [0]:
SELECT refund_id,
       payment_id,
       CAST(date_format(refund_timestamp, 'yyyy-MM-dd') AS DATE) AS refund_date,
       date_format(refund_timestamp, 'HH:mm:ss') AS refund_time,
       refund_amount,
       regexp_extract(refund_reason, '^([^:]+):', 1) AS refund_reason,
       regexp_extract(refund_reason, '^[^:]+:(.*)$', 1) AS refund_source
  FROM hive_metastore.bronze.refunds;

### 3. Write transformed data to the Silver schema  

In [0]:
CREATE SCHEMA hive_metastore.silver;

In [0]:
CREATE TABLE hive_metastore.silver.refunds AS
SELECT refund_id,
       payment_id,
       CAST(date_format(refund_timestamp, 'yyyy-MM-dd') AS DATE) AS refund_date,
       date_format(refund_timestamp, 'HH:mm:ss') AS refund_time,
       refund_amount,
       regexp_extract(refund_reason, '^([^:]+):', 1) AS refund_reason,
       regexp_extract(refund_reason, '^[^:]+:(.*)$', 1) AS refund_source
FROM hive_metastore.bronze.refunds;

In [0]:
SELECT * FROM hive_metastore.silver.refunds;

In [0]:
DESCRIBE EXTENDED hive_metastore.silver.refunds;